# Place bid & Calculate Profits

In [6]:
import numpy as np

### Sample Data

In [33]:
# sample list for now 
dam_list = [22.0, 20.0, 19.0, 18.0, 19.0, 23.0, 28.0, 32.0, 35.0, 38.0, 41.0, 44.0, 42.0, 39.0, 45.0, 55.0, 60.0, 58.0, 52.0, 40.0, 32.0, 28.0, 25.0, 23.0]
rtm_proj_list = [21.0, 19.0, 18.0, 19.0, 21.0, 25.0, 31.0, 35.0, 37.0, 40.0, 45.0, 47.0, 40.0, 42.0, 48.0, 62.0, 67.0, 59.0, 55.0, 38.0, 30.0, 26.0, 24.0, 22.0]
rtm_real_list = [20.5, 18.5, 17.5, 18.5, 20.5, 24.5, 30.5, 34.5, 36.5, 39.5, 44.5, 46.5, 39.5, 41.5, 47.5, 61.5, 66.5, 58.5, 54.5, 37.5, 29.5, 25.5, 23.5, 21.5]

In [34]:
# Ensure data lengths match
if len(dam_list) != len(rtm_proj_list) or len(dam_list) != len(rtm_real_list) or len(rtm_proj_list) != len(rtm_real_list):
    raise ValueError("Lists must have the same length.")    


### Test 1: No Scenarios or Scaling

In [114]:
# Set bid ranges 
SMALLEST_BID_1 = 1 # in MW 
LARGEST_BID_1 = 25 # in MW

TRADE_FLOOR_1 = 1 # don't bid under 1
TRADE_MAX_1 = 20 # make full bid over 20 

In [115]:
# Bid calculation function 
def calculate_bid_1(dam_price, rtm_price):
    spread = rtm_price - dam_price
    abs_spread = np.abs(spread)
    
    if abs_spread >= TRADE_FLOOR_1:
        faux_spread_percentile = min((abs_spread - TRADE_FLOOR_1) / (TRADE_MAX_1 - TRADE_FLOOR_1), 1.0)
        bid = SMALLEST_BID_1 + ((LARGEST_BID_1 - SMALLEST_BID_1) * faux_spread_percentile)
        if spread < 0: 
            bid *= -1
    else: 
        bid = 0

    return bid


### Test 2: No Scenarios, Scaling

In [121]:
# Set bid ranges 
SMALLEST_BID_2 = 1 # in MW 
LARGEST_BID_2 = 25 # in MW

LARGEST_SPREAD = 0 

# Train price spread thresholds
TRADE_FLOOR_2 = 1 # don't bid under 1
TRADE_MAX_2_BASE = 20 # make full bid over 20 

In [135]:
# Bid calculation function 
def calculate_bid_2(dam_price, rtm_price):
    global LARGEST_SPREAD
    
    spread = rtm_price - dam_price
    abs_spread = np.abs(spread)

    if LARGEST_SPREAD < abs_spread:
        LARGEST_SPREAD = abs_spread

    trade_floor_max = max(LARGEST_SPREAD, TRADE_MAX_2_BASE)

    if abs_spread >= TRADE_FLOOR_2:
        faux_spread_percentile = min((abs_spread - TRADE_FLOOR_2) / (trade_floor_max - TRADE_FLOOR_2), 1.0)
        bid = SMALLEST_BID_2 + ((LARGEST_BID_2 - SMALLEST_BID_2) * faux_spread_percentile)
        if spread < 0: 
            bid *= -1
    else: 
        bid = 0

    return bid


### Profit Calculations

#### Helper Function

In [136]:
# Calculate profits
def profit_calculator(dam_price, rtm_real_price, bids): 
    total_profit = 0
    for i in range(len(bids)):
        hourly_profit = 0
        bid = bids[i]
        day_ahead_price = dam_price[i]
        real_price = rtm_real_price[i]

        profit_per_mw = real_price - day_ahead_price
        hourly_profit += profit_per_mw * bid

        total_profit += hourly_profit

    return total_profit

#### Test 1

In [137]:
bids = []

for i in range(len(dam_list)):
    dam_price = dam_list[i]
    rtm_price = rtm_proj_list[i]
    bid = calculate_bid_1(dam_price, rtm_price)
    bids.append(bid) 

profit = profit_calculator(dam_list, rtm_real_list, bids)
print(profit)



225.89473684210523


#### Test 2

In [138]:
bids = []

for i in range(len(dam_list)):
    dam_price = dam_list[i]
    rtm_price = rtm_proj_list[i]
    bid = calculate_bid_2(dam_price, rtm_price)
    bids.append(bid) 

profit = profit_calculator(dam_list, rtm_real_list, bids)
print(profit)



225.89473684210523
